In [2]:
import numpy as np
word_embeddings = {}
f = open('C:/users/Videet/1/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [5]:
import tika,re
from tika import parser
tika.initVM()
file = '1.txt'
# Parse data from file
file_data = parser.from_file(file)
# Get files text content
article_text = file_data['content']

In [6]:
article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)  
article_text = re.sub(r'\s+', ' ', article_text) 
formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )  
formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)  

In [7]:
article_text

' Anurag Mohan Nair 302, Saga Tower, Plot No. 97, Sec – 27, Nerul (E), Navi Mumbai || E Mail - nair.anurag@outlook.com Phone: +91 7208850948 Page 1 of 2 Objective – Seeking a position in an IT company, so as to obtain a platform to portray my technical and inter-personal skills, while refining and developing my programming and creative abilities. Education ? University of Mumbai, Pillai’s College of Engineering Panvel, Maharashtra Third Year, Computer Science Engineering Current Overall GPA: 8.1 ? D.A.V Public School, Nerul Navi Mumbai, Maharashtra o HSC (CBSE) May 2014, 82.6% o SSC (CBSE) May 2012, 83.6% Relevant Technical Coursework & Projects ? Website Development- Created a website using HTML, CSS, JavaScript and PHP for a gaming forum where users can create their accounts and read about various new games in the market and make comments and talk with other users. ? Chat Application- Created a chat application using Socket Programming and Java. The application could run on multiple 

In [8]:
import nltk
sentence_list = nltk.sent_tokenize(article_text)  

In [9]:
import pandas as pd
clean_sentences = pd.Series(sentence_list).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]


In [10]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

In [11]:
sim_mat = np.zeros([len(sentence_list), len(sentence_list)])


In [12]:
from sklearn.metrics.pairwise import cosine_similarity
for i in range(len(sentence_list)):
    for j in range(len(sentence_list)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [13]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentence_list)), reverse=True)
for i in range(1):
    print(ranked_sentences[i][1])

97, Sec – 27, Nerul (E), Navi Mumbai || E Mail - nair.anurag@outlook.com Phone: +91 7208850948 Page 1 of 2 Objective – Seeking a position in an IT company, so as to obtain a platform to portray my technical and inter-personal skills, while refining and developing my programming and creative abilities.
